In [19]:
# used for manipulating directory paths
import os

# Scientific and vector computation for python
import numpy as np

import math

# SciKit-Learn
import sklearn as skl
from sklearn import svm

# Plotting library
from matplotlib import pyplot
import matplotlib as mpl

# Optimization module in scipy
from scipy import optimize


In [20]:
X_train = np.zeros((7868, 76))
Y_train = np.zeros((7868, 147))
X_test = np.zeros((2705, 76))
predictions = np.zeros((2705, 147))

# Load X_train
dir_string = '../data/train_feature_files'
training_files = os.listdir(dir_string)

for file in training_files:
    file_path = dir_string + '/' + file
    song_id_str = file_path.split('/')[-1].split('.')[0]
    a = np.load(file_path)
    a_summary = a['summary']
    X_train[int(song_id_str)] = a_summary
    
# Load Y_train
with open('../data/cal10k_train_data.csv', 'r') as y_train_file:
    for line in y_train_file:
        if 'id' not in line:
            features_w_id = line.strip().split(',')
            features_no_id = features_w_id[1:]
            for idx in range(len(features_no_id)):
                features_no_id[idx] = int(features_no_id[idx])
            Y_train[int(features_w_id[0])] = np.array(features_no_id)
            
# Load X_test
dir_string = '../data/test_feature_files/'
testing_files = os.listdir(dir_string)

for file in testing_files:
    file_path = dir_string + file
    song_id_str = file_path.split('/')[-1].split('.')[0]
    data = np.load(file_path)
    data_summary = data['summary']
    X_test[int(song_id_str)] = data_summary

In [ ]:
where_are_NaNs = np.isnan(X_train)
X_train[where_are_NaNs] = 0
where_are_NaNs = np.isnan(X_test)
X_test[where_are_NaNs] = 0
print(X_train.shape)
print(X_test.shape)

(7868, 76)
(2705, 76)


In [ ]:
genre_models = list()
for i in range(147):  # For each genre
    genre_model = svm.SVC(gamma='scale', probability=False)
    genre_model.fit(X_train, Y_train[:,i])
    genre_models.append(genre_model)
    
print(len(genre_models))

In [ ]:
for i in range(147):
    genre_prediction_score = genre_models[i].decision_function(X_test)
    print(genre_prediction_score[0])
    predictions[:, i] = genre_prediction_score[:,1]  # Get probability that it is class=1
        
print(predictions)


In [ ]:
file = open('../data/cal10k_test_random_submission.csv', 'r')
id_line = None
for line in file:
    if 'id' in line:
        id_line = line
ids = []
for i in range(2705):
    ids.append(int(i))
predictions = np.insert(predictions, 0, ids, axis=1)
string_predictions = []
for song_idx in range(len(predictions)):
    song = predictions[song_idx]
    song_string = []
    for genre_idx in range(len(song)):
        if genre_idx != 0:
            song_string.append(str(predictions[song_idx, genre_idx]))
        else: 
            song_string.append(str(predictions[song_idx, genre_idx]))
    string_predictions.append(song_string)
with open('../data/svms_are_fucking_cool.csv', 'w') as out_file:
    out_file.write(id_line)
    for song_array in string_predictions:
        song_array[0] = str(int(float(song_array[0])))
        out_file.write(",".join(song_array) + os.linesep)

In [12]:
half_X_train_len = math.floor(len(X_train)/2)
half_Y_train_len = math.floor(len(Y_train)/2)
X_cv_train = X_train[0:half_X_train_len]
Y_cv_train = Y_train[0:half_Y_train_len]
X_cv_test = X_train[half_X_train_len:]
Y_cv_test = Y_train[half_Y_train_len:]
cv_predictions = np.zeros(Y_cv_test.shape)

genre_models = list()
for i in range(147):  # For each genre
    genre_model = svm.SVC(gamma='scale', probability=False)
    genre_model.fit(X_cv_train, Y_cv_train[:,i])
    genre_models.append(genre_model)

for i in range(147):
    genre_prediction_prob = genre_models[i].predict(X_cv_test)
    cv_predictions[:, i] = genre_prediction_prob  # Get probability that it is class=1
        
print(cv_predictions)

[[ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 ..., 
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]]


In [13]:
print(cv_predictions.shape)
print(skl.metrics.roc_auc_score(Y_cv_test, cv_predictions))

(3934, 147)
0.50010975854
